# Starbucks Capstone Challenge 
## Prepare data 

In [1]:
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
    
sns.set_theme(style='darkgrid')
# read in the json files
portfolio = pd.read_json('../data/raw/portfolio.json', orient='records', lines=True)
profile = pd.read_json('../data/raw/profile.json', orient='records', lines=True)
transcript = pd.read_json('../data/raw/transcript.json', orient='records', lines=True)

### 1. Data processing

In [2]:
# Get offer ids from 'value' column, convert to float, and store in new column
offer_ids = dict()
indx = list(transcript[transcript['event']!='transaction'].index)

for ind in indx: 
    offer_id = list(transcript.iloc[ind]['value'].values())[0]
    offer_ids.update({ind:offer_id})
    
# Make dataframe from dictionary of index, offer_id strings     
offer_id_df = pd.DataFrame.from_dict(offer_ids, orient='index', columns=['offer_ids'])

# Concat transcript_mod and offer_id_df dataframes
transcript_mod = pd.concat([transcript, offer_id_df], axis=1, ignore_index=False)

In [3]:
# rename column 'id' as offer_ids to remain consistent with transcript df
portfolio = portfolio.rename(columns={'id':'offer_ids'})

In [4]:
# merge transcript and portfolio dataframes
transcript_portfolio = transcript_mod.merge(portfolio[['offer_ids', 'offer_type']], on='offer_ids', how='left')

# filter transcript_portfolio to get transcripts corresponding to BOGO offers 
bogo = transcript_portfolio[transcript_portfolio['offer_type']=='bogo']

,person,event,value,time,offer_ids,offer_type
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo
5,389bc3fa690240e798340f5a15918d5c,offer received,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},0,f19421c1d4aa40978ebb69ca19b0e20d,bogo
12,4b0da7e80e5945209a1fdddfe813dbe0,offer received,{'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'},0,ae264e3637204a6fb9bb56bc8210ddfd,bogo
18,ebe7ef46ea6f4963a7dd49f501b26779,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo
...,...,...,...,...,...,...
306413,8ea3ffcc740848f99c9679e3bc3c9c80,offer viewed,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},714,f19421c1d4aa40978ebb69ca19b0e20d,bogo
306421,89417e13fcd54dd78b80f97f7f085cf1,offer viewed,{'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'},714,ae264e3637204a6fb9bb56bc8210ddfd,bogo
306423,8771f96bc7f947c09a8f78ee2e14a1a7,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,714,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo
306430,0355c6a5fdbc429ea5b05e8a9ecd2eae,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,714,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo


In [5]:
# count the number of events per person
bogos_per_person = bogo.groupby('person')['event'].value_counts().unstack()
bogos_per_person

event,offer completed,offer received,offer viewed
person,,,
0009655768c64bdeb2e877511632db8f,1.0,1.0,1.0
00116118485d4dfda04fdbaba9a87b5c,NaN,2.0,2.0
0011e0d4e6b944f998e987f904e8c1e5,1.0,1.0,1.0
0020c2b971eb4e9188eac86d93036a77,1.0,2.0,1.0
0020ccbbb6d84e358d3414a3ff76cffd,2.0,2.0,2.0
...,...,...,...
fff3ba4757bd42088c044ca26d73817a,1.0,1.0,1.0
fff7576017104bcc8677a8d63322b5e1,1.0,3.0,2.0
fff8957ea8b240a6b5e634b6ee8eafcf,NaN,1.0,1.0


In [6]:
# rename column 'id' to 'person' to remain consistent with transcript
profile = profile.rename(columns={'id':'person'})

In [7]:
# Remove people was ages > 99
profile = profile[profile['age'] <= 99]

In [8]:
# Remove people with income > 1000000
profile = profile[profile['income'] < 1000000.0]

### 2. Feature engineering

In [9]:
# Binarise age based on criteria old=1, young=0
def age_group(age):
    if age > 35: 
        return 'old'
    else:
        return 'young'
profile['age_group'] = profile['age'].apply(lambda x:age_group(x))

In [10]:
# Binarise income based on criteria rich=1, poor=0
def income_group(income):
    if income > 60000: 
        return 'high income'
    else:
        return 'low income'
profile['income_group'] = profile['income'].apply(lambda x:income_group(x))

In [11]:
# select columns needed for training model
profile_subset = profile[['person','income_group','age_group', 'gender']]

In [12]:
# reset index to allow ease concatanation with transformed ohe data
profile_subset = profile_subset.reset_index()

profile_subset.drop('index', axis=1, inplace=True)

In [13]:
# One hot encode age_group, income_group, gender

# intantiate onehotencoder
enc = OneHotEncoder(handle_unknown='ignore')

# fit onehotencoder on data
enc.fit(profile_subset[['income_group','age_group', 'gender']])

# one hot encode features
transformed = enc.transform(profile_subset[['income_group','age_group', 'gender']]).toarray()

#Create a Pandas DataFrame of the hot encoded column
ohe_df = pd.DataFrame(transformed, columns=enc.get_feature_names_out())

#concat with original data
profile_subset_ohe = pd.concat([profile_subset, ohe_df], axis=1)

profile_subset_ohe.drop(['income_group','age_group', 'gender'], axis=1, inplace=True)

In [14]:
# merge bogo df with demographic data
bogos_per_person_demo = bogos_per_person.merge(profile_subset_ohe, left_index=True, right_on='person', how='left').set_index('person')

In [15]:
# add viewed and received columns and only keep people who received and viewed at least one bogo
bogos_per_person_demo['received'] = bogos_per_person_demo['offer received'] > 0
bogos_per_person_demo['viewed'] = bogos_per_person_demo['offer viewed'] > 0

In [16]:
# drop people with missing demographic data (note: people either have all demographic data filled in or none, so only need to drop based on missing demographic data in one column)
bogos_per_person_demo.dropna(subset=['age_group_old'], inplace=True)

In [17]:
# select columns needed for classifier
bogo_data = bogos_per_person_demo.drop(columns=['offer received','offer viewed', 'offer completed', 'received'], axis=1)
bogo_data.head()

,income_group_high income,income_group_low income,age_group_old,age_group_young,gender_F,gender_M,gender_O,viewed
person,,,,,,,,
0009655768c64bdeb2e877511632db8f,1.0,0.0,0.0,1.0,0.0,1.0,0.0,True
0011e0d4e6b944f998e987f904e8c1e5,0.0,1.0,1.0,0.0,0.0,0.0,1.0,True
0020c2b971eb4e9188eac86d93036a77,1.0,0.0,1.0,0.0,1.0,0.0,0.0,True
0020ccbbb6d84e358d3414a3ff76cffd,0.0,1.0,0.0,1.0,1.0,0.0,0.0,True
004b041fbfe44859945daa2c7f79ee64,1.0,0.0,1.0,0.0,1.0,0.0,0.0,True


In [18]:
# pickle prepared dataframe
bogo_data.to_pickle("../data/processed/bogo_data.pkl") 